# SVM ile metin siniflandirma

Bu calismada SVM ve daha once ogrendigimiz tasniflendirme algoritmalarini kullanarak bir anlik sohbet frubunda atilan mesajlarin belli bir kisiye ait olup olmadigini bulacak bir model gelistirecegiz.

### Kabiliyetler:

- SVM'i pekistirme ve diger modellerle kiyaslama
    - SVM parametre ayarlamasi
- regex
- Pipeline
- Metinler uzerinde model egitebilme

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Veriyi okuma

In [2]:
df = pd.read_csv("verbilimciler.txt", sep='\t')

FileNotFoundError: [Errno 2] No such file or directory: 'verbilimciler.txt'

In [ ]:
df.head()

,"[7/13/21, 12:02:14 PM] VeriBilimciler: ‎Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them."
0,"[7/13/21, 12:02:14 PM] ‎Abdullah Atçılı create..."
1,"[11/13/21, 12:31:53 AM] ‎Ersan Yagci added you"
2,"[11/13/21, 12:32:58 AM] ‎You're now an admin"
3,"[11/13/21, 5:38:02 AM] Ersan Yagci: Abiler sel..."
4,"[11/13/21, 5:38:38 AM] Ersan Yagci: Haftaya cu..."


In [ ]:
df.columns = [0]

In [ ]:
df = df.loc[3:]

In [ ]:
df

,0
3,"[11/13/21, 5:38:02 AM] Ersan Yagci: Abiler sel..."
4,"[11/13/21, 5:38:38 AM] Ersan Yagci: Haftaya cu..."
5,"[11/13/21, 5:39:20 AM] Ersan Yagci: Cumartesi ..."
6,"[11/13/21, 5:39:48 AM] Ersan Yagci: Biraz vite..."
7,"[11/13/21, 5:40:41 AM] Ersan Yagci: Şu ana kad..."
...,...
1296,"‎[1/1/22, 3:02:06 PM] Ahmet Umit Çalışkan: Imb..."
1297,"‎[1/1/22, 3:02:09 PM] Ahmet Umit Çalışkan: Chu..."
1298,"[1/1/22, 3:02:55 PM] Ahmet Umit Çalışkan: grid..."
1299,"[1/1/22, 3:09:41 PM] Sinan Yilmaz: Teşekkürler..."


## On islemler - biraz regexp

regexp ile yazi yiginindan tarih, saat, gonderen ve mesaj alanlarini ayirabiliriz.

In [ ]:
import re

In [ ]:
yazi = "22-06-2021 tarihinde yaptiginiz basvurunun..."

In [ ]:
re.findall('\d+-\d+-\d+', yazi)

['22-06-2021']

In [ ]:
re.sub('[0-9]', '', yazi)

'-- tarihinde yaptiginiz basvurunun...'

In [ ]:
re.sub('[a-z]', '', yazi)

'22-06-2021   ...'

In [ ]:
yazi = 'yollarda bulurum seni takvimlerden calarim seni dans ederim hayalinle'

In [ ]:
re.findall('\s\w+[urum|arim|erim]\s', yazi)

[' bulurum ', ' calarim ', ' ederim ']

Regex ile ilgili patern tanimlamalarini bulabileceginiz bir web sayfasi: https://docs.rackspace.com/support/how-to/regex-cheat-sheet-basics/

Bunun gibi pek cok "cheat sheet"'e cevrimici olarak erismek mumkundur.

In [ ]:
a = df[0].str

Mesaj zamaninin bulunmasi:

In [ ]:
datetime = a.findall("\[(\d+/\d+/\d+,\s\d+:\d+:\d+\s[AM|PM].*)\]").str[0]

In [ ]:
date = pd.to_datetime(datetime.str.split(',').str[0])

Gonderenin bulunmasi:

In [ ]:
sender = a.findall("\[\d+/\d+/\d+,\s\d+:\d+:\d+\s[AM|PM].*\](.*): ").str[0].str.strip()

Mesajin bulunmasi:

In [3]:
message = a.findall("\[\d+/\d+/\d+,\s\d+:\d+:\d+\s[AM|PM].*\].*: (.*)").str[0]

NameError: name 'a' is not defined

Dataframe olusturma

In [4]:
data = pd.DataFrame({'date': date, 'sender': sender, 'message': message})
data['y'] = (data['sender'] == 'Ersan Yagci').astype(int)
data['apo'] = (data['sender'] == 'Abdullah Atçılı').astype(int)
data = data.dropna(how='any')

NameError: name 'date' is not defined

Gurultuyu engellemek icin, bugune kadar 5'den fazla mesaj atmis kisileri bulup yalnizca bu kisilerin mesajlarini koruyalim:

In [531]:
kisiler = sender.value_counts().loc[lambda x: x>=5].index

In [532]:
data = data.loc[data['sender'].isin(kisiler)]

In [533]:
data

,date,sender,message,y,apo
3,2021-11-13,Ersan Yagci,Abiler selamlar,1,0
4,2021-11-13,Ersan Yagci,Haftaya cumartesi itibariyle machine learninge...,1,0
5,2021-11-13,Ersan Yagci,Cumartesi akşam 10da üstat Şükrü ilk dersi ver...,1,0
6,2021-11-13,Ersan Yagci,Biraz vites artırıcaz hem kod hem teori birlik...,1,0
7,2021-11-13,Ersan Yagci,Şu ana kadar olan kısmı kaçırmış olanlar bende...,1,0
...,...,...,...,...,...
1296,2022-01-01,Ahmet Umit Çalışkan,Imblearn_churn_modelling.ipynb ‎document omitted,0,0
1297,2022-01-01,Ahmet Umit Çalışkan,Churn_Modelling.csv ‎document omitted,0,0
1298,2022-01-01,Ahmet Umit Çalışkan,gridleri markdown yaptım. siz azaltıp kullanın...,0,0
1299,2022-01-01,Sinan Yilmaz,Teşekkürler kanka,0,0


### Pipeline

En az kodla en duzenli ve tekrarlanabilir deneyleri gerceklestirmek ve yapilanlarin organize kalmasini saglamak adina `sklearn`un pipeline ozelligi kullanilabilir.

**Simdiye kadar ne yapiyorduk?**

In [534]:
from sklearn import feature_extraction, svm, metrics, pipeline, preprocessing, model_selection, linear_model, neighbors

In [593]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(data['message'], data['y'], test_size=0.2,random_state=42, stratify=data['y'])

vectorizer = feature_extraction.text.CountVectorizer()
scaler = preprocessing.MaxAbsScaler()

f_train = vectorizer.fit_transform(X_train)
f_train_scaled = scaler.fit_transform(f_train)

model = svm.LinearSVC()

model.fit(f_train_scaled, y_train)

model.score(scaler.transform(vectorizer.transform(X_test)), y_test)  # anasinin gozu

0.919831223628692

**Daha sade bir ifade kullanabilir miydik: evet.**

```python
Pipeline(
    [
    ('adim1', donusturucu1),
    ('adim2', donusturucu2),
    ('adim3', donusturucu3),
    ...,
    ('tahminci', model)
       
    ]
)

```

In [594]:
boru = pipeline.Pipeline(
    [
        ('vektorcu', feature_extraction.text.CountVectorizer()),
        ('olcekci', preprocessing.MaxAbsScaler()),
        ('tahminci', svm.LinearSVC())
    ]
)

In [595]:
boru.fit(X_train, y_train)  # donusumleri acikca yazmiyoruz. ham veriyi pas etmek yeterli

Pipeline(steps=[('vektorcu', CountVectorizer()), ('olcekci', MaxAbsScaler()),
                ('tahminci', LinearSVC())])

In [596]:
boru.score(X_test, y_test)

0.919831223628692

Adimlari incelemek istersek:

- Kac kelime cikarmisiz?

In [597]:
len(boru.named_steps['vektorcu'].vocabulary_)

2282

- Buldugumnuz katsayilar neymis?

In [598]:
boru.named_steps['tahminci'].coef_

array([[-0.05336839, -0.02975786, -0.01665834, ..., -0.00135876,
         0.05691445, -0.05395648]])

## Pipeline, capraz dogrulama ile kullanilabilir mi?
Kesinlikle evet. Herhangi bir modelmiscesine kullabilabilir.

In [599]:
cv_sonucu = \
    model_selection.cross_validate(
        boru, data['message'], data['y'],
        cv=model_selection.StratifiedShuffleSplit(n_splits=30),
        return_train_score=True
        )

In [600]:
pd.DataFrame(cv_sonucu).agg(['mean', 'std'])

,fit_time,score_time,test_score,train_score
mean,0.020797,0.001934,0.910924,0.994178
std,0.009444,0.000785,0.021236,0.000968


Son adimin tum fonksiyonalitesi pipeline ve ham girdi uzerinden kullanilabilir.

In [601]:
boru.decision_function(['abiler gunaydin'])

array([0.17105798])

In [602]:
boru.predict(['abiler gunaydin'])

array([1])

In [611]:
boru.decision_function(['abi pandas'])

array([0.11815814])

In [612]:
boru.decision_function(['selam kanki'])

array([-1.21305397])

In [613]:
boru.decision_function(['beyler yavas gidiyoruz'])

array([-0.61513888])

## Sizin siraniz

- Ersan'in attigi mesajlari tespit edebilecek bir pipeline olusturabilir misiniz?
    - En az 3 farkli alternatif pipeline deneyip sonuclarinizi gosteriniz.
- Basari kistasi olarak hangi olcumleri kullanacaksiniz?
- Tek bir train-test cifti mi kullanacaksiniz yoksa modelinizi test etmek icin farkli dogrulama fikirleriniz var mi?
- Modelinizin parametrelerinin ne ise yaradigini kisaca tanimlar misiniz?
    - Bu problem icin en iyi calisan parametreler nelerdi? Nasil sectiniz?


In [571]:
data.head(10)

,date,sender,message,y,apo
3,2021-11-13,Ersan Yagci,Abiler selamlar,1,0
4,2021-11-13,Ersan Yagci,Haftaya cumartesi itibariyle machine learninge...,1,0
5,2021-11-13,Ersan Yagci,Cumartesi akşam 10da üstat Şükrü ilk dersi ver...,1,0
6,2021-11-13,Ersan Yagci,Biraz vites artırıcaz hem kod hem teori birlik...,1,0
7,2021-11-13,Ersan Yagci,Şu ana kadar olan kısmı kaçırmış olanlar bende...,1,0
8,2021-11-13,Ersan Yagci,Salı ve perşembe akşam 10da buluşalım bu hafta...,1,0
9,2021-11-13,‪Engin‬,Özür diliyorum sizden bayağı geride kaldım,0,0
10,2021-11-13,Ersan Yagci,"Ne özürü abi, başla ama Engin bu parta.",1,0
11,2021-11-13,‪Engin‬,Ama pandas ve sonrası kısımlar bayağı eksik bende,0,0
12,2021-11-14,Yusuf Can,https://signup.watchful.io/hand-labeling-is-ha...,0,0


In [572]:
# buradan devam ediniz